Al final hemos decidido hacer un split del dataset tal que en el conjunto de val y test hay 300 imágenes de cada clase, y en el conjunto de train hay 2123 elípticas, 2615 S0s y 7108 espirales.

En este code pretendo hacer el split y generar 3 csv's, train, val y test, respectivamente, con el número correcto de imágenes por clase, y de forma randomizada. 

La idea es meter 300 imágenes random de cada clase a val y test, y el resto a train.

In [35]:
import pandas as pd
import random

In [36]:
# Primero abro el csv mergeado
df = pd.read_csv("C:\\Users\\maseo\\OneDrive\\Escritorio\\GalaxyClassification\\Data\\Dataset\\path_TType.csv", sep=';')

# Ahora creo otra columna agrupando las subclases, de tal forma que solo haya 3 clases.

# Definir una función para asignar la clase final.
def assign_final_class(clase):
    if clase == -5:
        return 'Ellipticals'
    elif -3 <= clase <= 0:
        return 'S0s'
    elif 1 <= clase <= 9:
        return 'Spirals'
    else:
        return 'Irr-Misc'  # Si la clase no cumple con ninguna de las condiciones anteriores, puedes asignar otro valor

# Crear la nueva columna "Final_Class" basada en la columna de clase original
df['Clase'] = df['TType'].apply(assign_final_class)

# Ahora la idea es meter en val y test 300 imágenes de cada clase, pero aleatoriamente.

# Creo DataFrames vacíos para train, val y test
train_df = pd.DataFrame(columns=['#JID', 'fpCid', 'TType', 'Clase'])
val_df = pd.DataFrame(columns=['#JID', 'fpCid', 'TType', 'Clase'])
test_df = pd.DataFrame(columns=['#JID', 'fpCid', 'TType', 'Clase'])

# Definir el número de imágenes por clase en val y test
num_images_per_class = 300

# Iterar sobre cada clase
for clase in df['Clase'].unique():
    # Seleccionar las filas correspondientes a la clase actual
    clase_df = df[df['Clase'] == clase]
    
    # Barajar las filas aleatoriamente
    clase_df = clase_df.sample(frac=1).reset_index(drop=True)
    
    # Dividir las imágenes de la clase en val y test
    val_and_test = clase_df.head(num_images_per_class * 2)
    val_df = pd.concat([val_df, val_and_test.head(num_images_per_class)])
    test_df = pd.concat([test_df, val_and_test.tail(num_images_per_class)])
    
    # Las imágenes restantes van al conjunto de entrenamiento
    train_df = pd.concat([train_df, clase_df.iloc[num_images_per_class * 2:]])

# Guardar los DataFrames en archivos CSV
train_df.to_csv('C:\\Users\\maseo\\OneDrive\\Escritorio\\GalaxyClassification\\Data\\Dataset\\Train.csv', index=False, sep=';')
val_df.to_csv('C:\\Users\\maseo\\OneDrive\\Escritorio\\GalaxyClassification\\Data\\Dataset\\Val.csv', index=False, sep=';')
test_df.to_csv('C:\\Users\\maseo\\OneDrive\\Escritorio\\GalaxyClassification\\Data\\Dataset\\Test.csv', index=False, sep=';')


Aunque las Irr+Misc no nos interesan, para entender el split, en val y test hay 300 imágenes de Irr+Misc. ¿Cómo es posible? Realmente lo que hemos hecho ha sido barajar cada clase y coger un head de 600. Para cada val se han metido las primeras 300 y para test las ultimas 300. Como en Irr+Misc solo hay 388, hay valores repetidos. Luego en train no aparecen, obviamente, porque hemos metido en train el resto de esas 600 de la cabecera.